<a href="https://colab.research.google.com/github/Davis1233798/ANN_Data/blob/main/rclone_%E7%B2%BE%E7%B0%A1%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================= FORM ============================= #
#@markdown <font size=5>← Install rclone</font>
# ================================================================ #

import os
import IPython
from IPython.display import clear_output


!curl https://rclone.org/install.sh | sudo bash


try:
	os.makedirs("/root/.config/rclone", exist_ok=True)
except OSError as error:
	pass

clear_output()

In [ ]:
#@markdown 下載config
!git clone https://github.com/Davis1233798/rclone_config.git /root/.config/rclone/



In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← Mount/Unmount rclone Drive (Optional)</font>
# @markdown Mount a remote drive as a local drive on a mountpoint.
# @markdown ---
Cache_Directory = "DISK" #@param ["RAM", "DISK"]
# ================================================================ #

import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re

if not os.path.exists("/root/.ipython/mixlab.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://shirooo39.github.io/MiXLab/resources/mixlab.py \
                  -O /root/.ipython/mixlab.py"
  run(_spl(shellCmd))

from mixlab import (
    runSh,
    prepareSession,
    rcloneConfigurationPath,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Available drive to mount/unmount:</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))

prepareSession()
content = open(f"{rcloneConfigurationPath}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {rcloneConfigurationPath}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --buffer-size 256M' \
      ' --transfers 10' \
      ' --vfs-cache-mode full' \
      ' --vfs-cache-max-age 0h0m1s' \
      ' --vfs-cache-poll-interval 0m1s' \
      f' --cache-dir {cache_path}' \
      ' --allow-other' \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"The drive have been successfully mounted! - \t{mPoint}")
    else:
      print(f"Failed to mount the drive! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"The drive have been successfully unmounted! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)
MountCMD()
# ShowAC()

In [ ]:
import os
import pathlib
import zipfile
import re
import urllib.request
import requests
from IPython.display import HTML, clear_output
from urllib.parse import urlparse

# ==============================[ FORM ]==============================
#@markdown <font size="5">️← [Start] aria2 WebUI</font>
USE_FREE_TOKEN = True
REGION = "JP"
Aria2_rpc = True
Ariang_WEBUI = True
TOKEN = ""
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
# OUTPUT_DIR = ""
PORT = 8221
HOME = os.path.expanduser("~")
# ====================================================================

if not os.path.exists(f"{HOME}/.ipython/mixlab.py"):
    hCode = "https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/mixlab.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/mixlab.py")

from mixlab import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    CWD,
    displayUrl,
    findProcess,
    findPackageR
)

loadingAn()

runSh('apt install -y aria2')
pathlib.Path('ariang').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)

def latestTag(link):
    import re
    from urllib.request import urlopen
    htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
    return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]


if not os.path.exists("ariang/index.html"):
    # BASE_URL = r"https://github.com/mayswind/AriaNg"
    # LATEST_TAG = latestTag(BASE_URL)
    # urlF = f'{BASE_URL}/releases/download/{LATEST_TAG}/' \
    #             f'AriaNg-{LATEST_TAG}-AllInOne.zip'
    urllib.request.urlretrieve(
        findPackageR('mayswind/AriaNg', 'AllInOne.zip'),
        'ariang/new.zip'
    )
    with zipfile.ZipFile('ariang/new.zip', 'r') as zip_ref:
        zip_ref.extractall('ariang')
    try:
        pathlib.Path('ariang/new.zip').unlink()
    except FileNotFoundError:
        pass


try:
    if not OUTPUT_DIR:
        OUTPUT_DIR = f"downloads/"
    elif not os.path.exists(OUTPUT_DIR):
        clear_output()
        print("[ERROR!]: Unable to find path specified!")
        exx()
except:
    OUTPUT_DIR = f"{CWD}/downloads/"

if Aria2_rpc:
    if not findProcess("aria2c", "--enable-rpc"):
        try:
            trackers = requests.get(
                "https://trackerslist.com/best_aria2.txt"
            ).text
            cmdC = r"aria2c --enable-rpc --rpc-listen-port=6800 -D " \
                fr"-d {OUTPUT_DIR} " \
                r"-j 20 " \
                r"-c " \
                fr"--bt-tracker={trackers} " \
                r"--bt-request-peer-speed-limit=0 " \
                r"--bt-max-peers=0 " \
                r"--seed-ratio=0.0 " \
                r"--max-connection-per-server=10 " \
                r"--min-split-size=10M " \
                r"--follow-torrent=mem " \
                r"--disable-ipv6=true " \
                r"--user-agent=Transmission/2.77 --peer-agent=Transmission/2.77 " \
                r"--peer-id-prefix=-TR2770- " \
                r"--split=20 " \
                r" &"
            runSh(cmdC, shell=True)
        except:
            print("aria2 RPC cannot be started! Please try to re-run the cell.")

clear_output()

if Aria2_rpc:
    Server = PortForward_wrapper(
        PORT_FORWARD,
        TOKEN,
        USE_FREE_TOKEN,
        [['Aria2_rpc', 6800, 'http']],
        REGION.lower(),
        [f"{HOME}/.ngrok2/aria2.yml", 5042]
    )
    data = Server.start('Aria2_rpc', displayB=False)
    Host = urlparse(data['url']).hostname
    port = "80"

    clear_output()

if Ariang_WEBUI:
    if Aria2_rpc:
        filePath = 'ariang/index.html'
        with open(filePath, 'r+') as f:
            read_data = f.read()
            f.seek(0)
            f.truncate(0)
            read_data = re.sub(r'(rpcHost:"\w+.")|rpcHost:""', f'rpcHost:"{Host}"', read_data)
            read_data = re.sub(r'protocol:"\w+."', r'protocol:"ws"', read_data)
            read_data = re.sub(r'rpcPort:"\d+."', f'rpcPort:"{port}"', read_data)
            f.write(read_data)
        try:
            urllib.request.urlopen(f"http://localhost:{PORT}")
        except:
            runSh(f"python3 -m http.server {PORT} &", shell=True, cd="ariang/")
  
    Server = PortForward_wrapper(
        PORT_FORWARD,
        TOKEN,
        USE_FREE_TOKEN,
        [['Ariang', PORT, 'http']],
        REGION.lower(),
        [f"{HOME}/.ngrok2/ariang.yml", 5043]
    )
    data2 = Server.start('Ariang', displayB=False)
    data2['url'] = urlparse(data2['url'])._replace(scheme='http').geturl()
    displayUrl(data2, pNamU='ariaNg : ')

if Aria2_rpc:
    display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 100%;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>aria2 RPC Configuration<p><thead>        <tr>            <th>Protocol</th>            <th>Host</th>            <th>Port</th>        </tr>    </thead>    <tbody>        <tr>            <td>WebSocket</td><td>"""+Host+"""</td><td>"""+port+"""</td></tr></tbody></table><a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/aria2RpcConfigSetup.gif">How to use this configuration: [Click ME]</a></button><center>"""))


In [ ]:
# ============================= FORM ============================= #
# @markdown <font size=5>← [Start] Colab Stay Alive</font>
# @markdown > This cell runs a JS code that will automatically press the reconnect button when you got disconnected due to idle.
# ================================================================ #

import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Clicked on the connect button"); 
     btn.click() 
     }
   
   btn = document.getElementById('connect')
   if (btn != null){
     console.log("Clicked on the reconnect button"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

clear_output()

In [ ]:
# ==============================#============================== #
# @markdown <font size="4">Run this cell if there is no methods working in Method 1.</font>
# @markdown > If you want to run other cell, stop this first and then run it again if you need it.
# ==============================#============================== #

import time
from IPython.display import clear_output


def loop():
    counter = 0
    second = 60

    while True:
        clear_output()

        print(f'This cell has been running for {counter} minute(s)...')

        counter += 1
        time.sleep(second)


if __name__ == '__main__':
    try:
        loop()
    except KeyboardInterrupt:
        clear_output()
        exit
    else:
        pass


This cell has been running for 2 minute(s)...
